In [ ]:
# ============ IMPORTS ============
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

mlflow.set_experiment("bitcoin_near_peak_classification")
mlflow.sklearn.autolog()  # auto-log params, metrics, model, etc.

# ============ LOAD LABELLED DATA ============
labelled_df = pd.read_csv("data_cache/labelled_df.csv", index_col=0, parse_dates=True)

# Define X, y
target_col = "Near_Peak"
drop_cols = [target_col, "Close"]  # keep Close only as reference, not feature

feature_cols = [c for c in labelled_df.columns if c not in drop_cols]
X = labelled_df[feature_cols]
y = labelled_df[target_col]

# ============ SPLIT DATA ============
split_idx = int(len(labelled_df) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

# =========== DEFINE MODELS ============
models = {
    "log_reg": LogisticRegression(max_iter=1000),
    "random_forest": RandomForestClassifier(n_estimators=200, max_depth=8, random_state=42),
    "svm_rbf": SVC(kernel="rbf", probability=True, random_state=42),
}

# ============ TRAIN MODELS ============
for name, model in models.items():
    with mlflow.start_run(run_name=name):
        # Train model
        model.fit(X_train, y_train)
        
        # Log the trained model with signature
        signature = infer_signature(X_train, model.predict(X_train))
        model_info = mlflow.sklearn.log_model(
            model, 
            artifact_path="model",
            signature=signature
        )

In [ ]:
# ============ EVALUATE MODELS ============
results = []
for name, model in models.items():
    with mlflow.start_run(run_name=name):
        # Create evaluation dataset (X_test + targets)
        eval_data = X_test.copy()
        eval_data["Near_Peak"] = y_test
        
        # Evaluate with MLflow - generates all metrics & plots automatically
        eval_result = mlflow.models.evaluate(
            model=model_info.model_uri,
            data=eval_data,
            targets="Near_Peak",
            model_type="classifier",
            evaluator_config={
                "log_explainer": True,  # SHAP feature importance
                "explainer_type": "exact"
            }
        )
        
        # Collect key metrics for comparison table
        results.append({
            "Model": name,
            "Accuracy": f"{eval_result.metrics['accuracy_score']:.3f}",
            "Precision": f"{eval_result.metrics['precision_score']:.3f}",
            "Recall": f"{eval_result.metrics['recall_score']:.3f}",
            "F1-Score": f"{eval_result.metrics['f1_score']:.3f}",
            "ROC-AUC": f"{eval_result.metrics['roc_auc']:.3f}",
        })

In [ ]:
# ============ DISPLAY RESULTS ============
results_df = pd.DataFrame(results)
print(results_df.to_markdown(index=False))
results_df.to_csv("data_cache/model_performance.csv", index=False)
# Also log the comparison table as an artifact in a summary run
with mlflow.start_run(run_name="model_comparison_summary"):
    results_df.to_csv("data_cache/model_performance.csv", index=False)
    mlflow.log_artifact("data_cache/model_performance.csv")